# Nuclio - prediction function

## Setup the environment

In [2]:
# nuclio: ignore
import nuclio

### Set environment variables

In [3]:
import os

In [4]:
# nuclio: ignore
os.environ['KV_TABLE'] = os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'demos/face-recognition/artifacts/encodings')

In [5]:
# Iguazio access
%nuclio env FRAMESD=${V3IO_FRAMESD}
%nuclio env V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env KV_TABLE=${KV_TABLE}


%nuclio: setting 'FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'KV_TABLE' environment variable


In [6]:
### Set cron trigger

In [7]:
%nuclio config spec.triggers.secs.kind = "cron"
%nuclio config spec.triggers.secs.attributes.interval = "1h"



%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '1h'


In [8]:
%nuclio mount /User users/iguazio

mounting volume path /User as users/iguazio


In [9]:
### Base image

In [10]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Installations

When installing packages while working, Please reset the kernel to allow Jupyter to load the new packages.

In [11]:
%%nuclio cmd -c
pip install v3io_frames

### Imports

In [12]:
# Util
import v3io_frames as v3f
from datetime import datetime, timedelta
import os
import shutil

# DB
import v3io_frames as v3f

## Function code

In [13]:
def handler(context, event):
    kv_table_path = os.environ['KV_TABLE']
    v3io_access_key = os.environ['V3IO_ACCESS_KEY']
    client = v3f.Client("framesd:8081", token = v3io_access_key, container="users") 
    df = client.read(backend='kv', table=kv_table_path, reset_index=True) 
    context.logger.info(df.head())  
    df2 = df[['fileName', 'camera', 'label', 'imgUrl']]    
    options = ['unknown']
    df3 = df2[df2.fileName.str.startswith(tuple(options))]    
    for idx in range(len(df3)):
        img_url = df3.iloc[idx]['imgUrl']
        splited = img_url.split("/")
        destination = "/".join((splited[0],splited[1],splited[2],splited[3],"dataset/label_pending"))
        print(img_url)
        print(idx)
        print(splited)
        print(destination)
        # Move the content of
        # source to destination

        dest = shutil.move(img_url, destination)

In [14]:
%nuclio deploy -n unknown-labeling -p default -c

[nuclio.deploy] 2019-11-26 10:16:39,373 (info) Building processor image
[nuclio.deploy] 2019-11-26 10:16:48,514 (info) Build complete
[nuclio.deploy] 2019-11-26 10:16:54,603 (info) Function deploy complete
[nuclio.deploy] 2019-11-26 10:16:54,614 done creating unknown-labeling, function address: 52.247.24.73:32599
%nuclio: function deployed
